# Práctico Clase 1

NOMBRE ALUMNO: **SEBASTIAN LATORRE DIAZ**

Diplomado en Machine Learning Aplicado UC - 2024

**Profesor:** Vicente Dominguez

En este práctico compararemos los resultados de algoritmos de recomendación:
- Most popular.
- User-based KNN.
- Item-based KNN.

Utilizaremos la librería **surprise** (https://surpriselib.com/)


## Configuración inicial

In [ ]:
# descarga de datasets de train, test e información de items
!gdown 1gmOrtPpZpHJ0HeBwtne-kA8Bll4rFWW7
!gdown 1bnLJUEIRx13k4nxN7x7Fa-3L37rXre73
!gdown 1i92TtKsgf_3ffef8EVLH9NNArxvF-cMo

Downloading...
From: https://drive.google.com/uc?id=1gmOrtPpZpHJ0HeBwtne-kA8Bll4rFWW7
To: /content/u.item
100% 236k/236k [00:00<00:00, 28.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bnLJUEIRx13k4nxN7x7Fa-3L37rXre73
To: /content/u2.base
100% 1.58M/1.58M [00:00<00:00, 41.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i92TtKsgf_3ffef8EVLH9NNArxvF-cMo
To: /content/u2.test
100% 395k/395k [00:00<00:00, 13.1MB/s]


vemos los nombres de los archivos descargados:

In [ ]:
ls

sample_data/  u2.base  u2.test  u.item


instalacion e importacion de librerias:

In [ ]:
# instalacion de libreria surprise
!pip3 install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357225 sha256=c9bb3a8b5a4045d951c05f120854404f971f8cd808b03350d142a590ebf78c3d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import NormalPredictor # random rating prediction
from surprise import KNNWithMeans # user-based KNN y item-based KNN
from surprise.accuracy import rmse
import seaborn as sns
from surprise.model_selection import cross_validate

%matplotlib inline
sns.set(style="whitegrid")

## Análisis exploratorio de datos

### Datos de entrenamiento:

In [ ]:
df_train = pd.read_csv('u2.base',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)
df_train.head()

,userid,itemid,rating,timestamp
0,1,3,4,878542960
1,1,4,3,876893119
2,1,5,3,889751712
3,1,6,5,887431973
4,1,7,4,875071561


### Datos de test:
- Se tienen que repetir los usuarios y los items del set de entrenamiento.
- La tarea es precedir el rating de estos items.

In [ ]:
df_test = pd.read_csv('u2.test',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)
df_test.head()

,userid,itemid,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,8,1,875072484
3,1,9,5,878543541
4,1,21,1,878542772


Nombres de películas y metadata:

In [ ]:
# Cargamos el dataset con los items
columns = ['movieid', 'title', 'release_date', 'video_release_date', \
           'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

df_items = pd.read_csv('u.item',
                        sep='|',
                        index_col=0,
                        names = columns,
                        header=None,
                        encoding='latin-1')

# reset index
df_items = df_items.reset_index()

# diccionario que entrega nombre de la pelicula con el id
dict_item_title = {idx:title for idx, title in zip(df_items.movieid, df_items.title)}

df_items.head()

,movieid,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Estadísticas de ratings:

In [ ]:
df_train.describe()[['rating']]

,rating
count,80000.000000
mean,3.526463
std,1.124429
min,1.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,5.000000


#Preguntas

##1. **COMENTE LOS RESULTADOS (1 PTO):**
**RESPONDE AQUI.**


*   Tenemos 943 usuarios, y 1648 items (Peliculas)
*   La media de interacciones por item es de 48.54
*   La desviacion estandar es de 64.95 lo cual sugiere una distribución amplia en la popularidad de los items
*   El valor minimo de interacciones es 1
*   El 25% de las peliculas tine un valor igual o menor que 6
*   El 50% de las peliculas tine un valor igual o menor que 22
*   El 75% de las peliculas tine un valor igual o menor que 65
*   El valor maximo de interacciones es de 461
*   De acuerdo a toda esta información podemos deducir que las interacciones tienen una distribución de "long tail". Donde una menor cantidad de peliculas tienen interaciones muy altas mientras que otras peliculas pueden ser muy poco vistas y tener muy pocas interacciones tambien.
*   Esto implica que una minoría de items concentra la mayoría de las interacciones, caracterizando una distribución desigual de popularidad entre los contenidos.

Cuantos usuarios e items distintos tenemos en el set de entrenamiento:

In [ ]:
print("Users:", df_train["userid"].nunique())
print("Items:", df_train["itemid"].nunique())

Users: 943
Items: 1648


Número de interacciones por item:

In [ ]:
items = df_train.groupby(["itemid"]).count()['userid']
items.describe()

count    1648.000000
mean       48.543689
std        64.950707
min         1.000000
25%         6.000000
50%        22.000000
75%        65.000000
max       461.000000
Name: userid, dtype: float64

##2. **COMENTE LOS RESULTADOS (1 PTO):**

**RESPONDER AQUI**

*   Peliculas mas populares: Los items listados muestran los más populares en términos de interacciones. Por ejemplo, "Star Wars (1977)" encabeza la lista con 461 interacciones, seguido por "Fargo (1996)" con 414 interacciones y "Contact (1997)" con 409 interacciones, entre otros.

*   Diversidad de Géneros: La lista incluye una variedad de géneros cinematográficos, desde ciencia ficción como "Star Wars" hasta comedias como "Liar Liar (1997)" y películas de acción como "Independence Day (ID4) (1996)". Esto nos induce a decir que hay interés y consumo de diferentes tipos de contenido por parte de los usuarios.

*   Impacto en Recomendaciones: Estos items altamente populares podrían tener un impacto significativo en los algoritmos de recomendación, ya que tienden a recibir más atención y podrían influir más en las sugerencias automáticas a los usuarios.

*   Identificar estas peliculas puede ser útil para diseñar estrategias de promoción y marketing, para mantener su popularidad como tambien para explorar cómo promover otros items menos vistos pero con potencial.

Item con más interacciones (más populares):

In [20]:
most_active_items = items.sort_values(ascending=False)

most_active_items = most_active_items.to_frame().reset_index().rename(columns={"userid": "interactions"})

most_active_items['title'] = [dict_item_title[x] for x in most_active_items.itemid]

most_active_items.head(10)

,itemid,interactions,title
0,50,461,Star Wars (1977)
1,100,414,Fargo (1996)
2,258,409,Contact (1997)
3,181,406,Return of the Jedi (1983)
4,294,402,Liar Liar (1997)
5,286,397,"English Patient, The (1996)"
6,288,360,Scream (1996)
7,1,358,Toy Story (1995)
8,121,351,Independence Day (ID4) (1996)
9,300,345,Air Force One (1997)


##3. **COMENTE LOS RESULTADOS (1 PTO):**

RESPONDER AQUI

Basado en el análisis de la concentración de interacciones en los usuarios más activos, podemos hacer las siguientes observaciones:

*   **Concentración de Interacciones**: El 46.16% de todas las interacciones registradas provienen de los usuarios más activos, que representan el 20% del total de usuarios de nuestro conjunto de datos.

*   **Implicaciones de Distribución "Long Tail"**: Este resultado nos confirma la idea de una distribución "long tail" en nuestros datos, donde un grupo relativamente pequeño de usuarios contribuye significativamente a la mayoría de las interacciones. Esto es consistente con la observación previa de que algunas peliculas que son mucho más populares que otras

*   **Relevancia para Estrategias de Personalización**: Dado que una proporción significativa de interacciones proviene de estos usuarios activos, estrategias como la personalización y la recomendación podrían beneficiarse al enfocarse en satisfacer las preferencias de este segmento de usuarios.

*   **Consideraciones Estratégicas**: Es importante considerar cómo optimizar el servicio y la experiencia del usuario para estos usuarios más activos, pero al mismo tiempo que se busca como aumentar la participación y satisfacción de una gama más amplia de usuarios menos activos.

Concentración de interacciones en 20% de usuarios más activos:

In [ ]:
proportion = 0.2
N = int(df_train.userid.nunique() * proportion)
interactions_by_active_items = most_active_items[:N]['interactions'].sum() / items.sum() * 100
print("{:.2f}% de las interacciones viene de los {} usuarios más activos ({:.0f}%)".format(
    interactions_by_active_items, N, proportion*100))

46.16% de las interacciones viene de los 188 usuarios más activos (20%)


##4. **COMENTE LOS RESULTADOS (1 PTO):**

RESPONDER AQUI

**NormalPredictor (Predicción Aleatoria):**
*   Se utilizó el modelo NormalPredictor, que predice los ratings de manera aleatoria basado en la distribución observada en los datos.
*   El RMSE obtenido fue 1.5217, esto nos sirve como punto de referencia para comparar la precisión de otros modelos mas sofisticados.

**KNN-User-Based (Vecinos más Cercanos basado en Usuarios):**

*   Se empleó el modelo KNNWithMeans con configuración para calcular similitud basada en usuarios (user_based=True) usando distancia coseno.
*   El RMSE obtenido fue de 0.9889, indica una mejora significativa en comparación con el modelo de predicción aleatoria. Esto nos dice que considerar la similitud entre usuarios para hacer predicciones puede ser más efectivo que simplemente hacer predicciones al azar.

**KNN-Item-Based (Vecinos más Cercanos basado en Ítems):**

*   Se utilizó el modelo KNNWithMeans configurado para calcular similitud basada en ítems (user_based=False) usando distancia coseno.
*   El RMSE obtenido fue ligeramente mejor 0.9839, comparado con el modelo basado en usuarios. Esto nos dice que considerar la similitud entre ítems para hacer predicciones puede ser aún más efectivo que el enfoque basado en usuarios en este conjunto de datos específico.

**Resultados**

*   El modelo NormalPredictor sirve como una línea base simple pero es poco precisa.
*   Los modelos de filtrado colaborativo basados en vecinos más cercanos (en ambos casos, usuario como ítem) mejoran significativamente la precisión de las predicciones de ratings en comparación con la predicción aleatoria.
*   En este caso particular, el modelo basado en ítems parece proporcionar las predicciones más precisas según el RMSE obtenido, superando al modelo basado en usuarios y al predictor aleatorio.


## Convertir dataframe de Pandas a formato surprise

In [ ]:
reader = Reader(rating_scale=(1, 5))
data_train = Dataset.load_from_df(df_train[['userid', 'itemid', 'rating']], reader)
data_test = Dataset.load_from_df(df_test[['userid', 'itemid', 'rating']], reader)

# procesar data para libreria surprise
data_train = data_train.build_full_trainset()
data_test = [data_test.df.loc[i].to_list() for i in range(len(data_test.df))]


## Rating Aleatorio
- En surprise: `NormalPredictor`

In [ ]:
algo_rndm = NormalPredictor()
algo_rndm.fit(data_train)
predictions = algo_rndm.test(data_test)
RMSE = rmse(predictions)

RMSE: 1.5217


## Prediccion de rating utilizando informacion de Usuarios más cercanos (KNN-user-based):


In [ ]:
sim_options = { 'name': 'cosine' ,'user_based':  True}

algo_knn_user = KNNWithMeans(k = 10 , sim_options = sim_options) # modificar cantidad de vecinos parametro K

algo_knn_user.fit(data_train)
predictions = algo_knn_user.test(data_test)
RMSE = rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9889


## Prediccion de rating utilizando informacion de Items más cercanos (KNN-item-based):


In [ ]:
sim_options = { 'name': 'cosine' ,'user_based':  False} # item-based CF

algo_knn_item = KNNWithMeans(k = 10 , sim_options = sim_options) # modificar cantidad de vecinos parametro K

algo_knn_item.fit(data_train)
predictions = algo_knn_item.test(data_test)
RMSE = rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9839


## 5. COMENTE LOS RESULTADOS DE TODOS LOS RECOMENDADORES.

Considere que métrica de error RMSE, donde mientras menores son más acertadas son las recomendaciones.

- ¿Cuál es el algoritmo que obtiene mejor desempeño en terminos de RMSE?


*   NormalPredictor (Predicción Aleatoria): Obtuvo un RMSE de 1.5217.
*   KNN-User-Based (Vecinos más Cercanos basado en Usuarios): Obtuvo un RMSE de 0.9889.
*   KNN-Item-Based (Vecinos más Cercanos basado en Ítems): Obtuvo un RMSE de 0.9839.

**Análisis de los Resultados:**

*   El algoritmo que obtiene el mejor desempeño en términos de RMSE, lo que quiere decir es que tiene las predicciones más precisas según esta métrica, es el KNN-Item-Based. Este modelo utiliza la similitud entre items para hacer recomendaciones, logró un RMSE ligeramente inferior al del modelo KNN-User-Based y significativamente mejor que el modelo NormalPredictor.Esto quiere decir que considerar la similitud entre ítems en lugar de entre usuarios resulta más efectivo para hacer predicciones de ratings en este conjunto de datos específico.

Por lo tanto, en base a la métrica RMSE, el KNN-Item-Based se destaca como el algoritmo que ofrece el mejor desempeño entre los tres evaluados.

**RESPONDER AQUI (3 PTOS):**

## 6. Análisis de sensibilidad de User-Based KNN (5 PTOS)
- Modificar cantidad de USUARIOS vecinos (K) a 5, 10, 15, 20 , 30 , 50, 60, 70,80, 90 y 100 e imprimir el valor de RMSE para cada uno.


In [21]:
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise.accuracy import rmse
import pandas as pd

# Supongamos que tienes tus datos en un DataFrame df_train con columnas 'userid', 'itemid' y 'rating'

# Paso 1: Cargar los datos en formato Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_train[['userid', 'itemid', 'rating']], reader)

# Paso 2: Definir los valores de K a evaluar
k_values = [5, 10, 15, 20, 30, 50, 60, 70, 80, 90, 100]

# Paso 3: Iterar sobre los valores de K y calcular RMSE para cada uno
for k in k_values:
    # Configurar el algoritmo KNN-User-Based con K vecinos
    sim_options = {'name': 'cosine', 'user_based': True}
    algo_knn_user = KNNWithMeans(k=k, sim_options=sim_options)

    # Evaluar el modelo usando validación cruzada
    results = cross_validate(algo_knn_user, data, measures=['RMSE'], cv=5, verbose=False)

    # Obtener el RMSE promedio de todas las particiones
    rmse_mean = round(results['test_rmse'].mean(), 4)

    # Imprimir resultados
    print(f"K = {k}: RMSE = {rmse_mean}")




Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
K = 5: RMSE = 1.0354
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
K = 10: RMSE = 0.9926
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Don

**7.COMENTE LOS RESULTADOS AQUI SOBRE LA CANTIDAD OPTIMA DE VECINOS (USUARIOS) CERCANOS INDIQUE INTUICIÓN DE POR QUE PASADO CIERTA CANTIDAD DE USUARIOS VECINOS EL RENDIMIENTO COMIENZA A DECRECER?** (5 PTOS)

RESPONDER AQUI.

Basado en los resultados obtenidos del algoritmo User-Based KNN variando la cantidad de vecinos (K), podemos hacer las siguientes observaciones:

**RMSE en función de K**: Observamos que a medida que incrementamos el número de usuarios vecinos (K), el valor de RMSE tiende a disminuir inicialmente. Sin embargo, después de cierto punto, aproximadamente a partir de K =30, la mejora en el RMSE se vuelve más gradual e incluso puede estabilizarse en k =70, en k =90 incluso aumentar ligeramente.

**Óptimo de Vecinos**: Según los resultados obtenidos, el valor más bajo de RMSE se alcanza aproximadamente en K = 70 y K = 80, donde ambos tienen un RMSE de 0.9596. Esto sugiere que utilizar entre 70 y 80 vecinos parece ser óptimo para este conjunto de datos específico y esta configuración de modelo.

**Intuición sobre el Decrecimiento del Rendimiento:** Pasado cierta cantidad de usuarios vecinos, el rendimiento comienza a decrecer posiblemente por varias razones:

1.   **Diversidad en las Preferencias**: A medida que aumenta K, se incluyen usuarios con preferencias más diversas y probablemente menos similares, lo que puede diluir la relevancia de los vecinos cercanos en la predicción para un usuario dado.
2.   **Ruido y Varianza**: Más vecinos pueden introducir más ruido o varianza en las estimaciones de ratings, especialmente si los usuarios vecinos tienen historiales de comportamiento muy diferentes o están menos correlacionados con el usuario objetivo.

Para concluir seleccionar un número adecuado de vecinos cercanos (en este caso, entre 70 y 80) puede mejorar significativamente la precisión del modelo User-Based KNN para la predicción de ratings.